In [59]:
import laspy
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon

# Airborne LiDAR tile query

In [28]:
pc_tiles = gpd.read_file("Coastal_NJ/2014_NGS_postSandy_topobathy_njtiles_index.shp")
pc_tiles

,Index,Name,URL,block,geometry
0,21687,Block_101/20140316_529000e_4339000n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_101,"POLYGON ((-74.66417 39.19525, -74.66417 39.199..."
1,21688,Block_101/20140316_529000e_4338500n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_101,"POLYGON ((-74.66419 39.19075, -74.66419 39.195..."
2,21689,Block_101/20140316_529000e_4336000n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_101,"POLYGON ((-74.66429 39.16932, -74.66429 39.172..."
3,21690,Block_101/20140316_528500e_4340500n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_101,"POLYGON ((-74.66990 39.20879, -74.66990 39.213..."
4,21691,Block_101/20140316_528500e_4340000n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_101,"POLYGON ((-74.66992 39.20428, -74.66992 39.208..."
...,...,...,...,...,...
12150,39685,Block_091/20131202_473000e_4352500n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_091,"POLYGON ((-75.31323 39.31698, -75.31323 39.321..."
12151,39686,Block_091/20131202_473000e_4352000n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_091,"POLYGON ((-75.31321 39.31248, -75.31321 39.317..."
12152,39687,Block_091/20131202_473000e_4351500n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_091,"POLYGON ((-75.31319 39.30797, -75.31319 39.312..."
12153,39688,Block_091/20131202_473000e_4351000n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_091,"POLYGON ((-75.31317 39.30346, -75.31317 39.307..."


In [31]:
pc_tiles = pc_tiles.to_crs(epsg="32618")

In [33]:
pc_tiles.head()

,Index,Name,URL,block,geometry
0,21687,Block_101/20140316_529000e_4339000n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_101,"POLYGON ((529000.049 4338498.137, 528998.190 4..."
1,21688,Block_101/20140316_529000e_4338500n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_101,"POLYGON ((529000.044 4337998.121, 528998.186 4..."
2,21689,Block_101/20140316_529000e_4336000n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_101,"POLYGON ((529000.080 4335620.374, 528998.675 4..."
3,21690,Block_101/20140316_528500e_4340500n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_101,"POLYGON ((528500.068 4339998.161, 528498.240 4..."
4,21691,Block_101/20140316_528500e_4340000n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_101,"POLYGON ((528500.024 4339498.155, 528498.196 4..."


In [6]:
pc = laspy.read("SandyNE_NJ_tileGrid/tr_0618p1_split.las")
header = pc.header
extent = [*header.min, *header.max]

In [73]:
x_min = extent[0]
x_max = extent[3]
y_min = extent[1]
y_max = extent[4]

pc_envelope = Polygon([[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]])
df_envelope = gpd.GeoDataFrame(index=[0], crs=pc_tiles.crs, geometry=[pc_envelope])

In [38]:
gpd.sjoin(df_envelope, pc_tiles.to_crs(epsg="32618"), how='inner')

,geometry,index_right,Index,Name,URL,block
0,"POLYGON ((579050.000 4436590.000, 579270.000 4...",5103,29664,Block_112/20121214_579000e_4437000n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_112


In [39]:
gpd.sjoin(pc_tiles.to_crs(epsg="32618"), df_envelope, how='inner')

,Index,Name,URL,block,geometry,index_right
5103,29664,Block_112/20121214_579000e_4437000n.laz,ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z...,Block_112,"POLYGON ((579000.095 4436494.780, 578994.836 4...",0


# Ground Control

In [51]:
file = open("0618-P1-NJSPSF.txt", 'r')
path = file.readlines()
file.close()

head = path[31].strip().replace("(", "").replace(")", "").split(",")
data  = path[33:]

In [52]:
head

['time in Sec',
 ' distance in Meters',
 ' position in US Survey Feet',
 ' lat',
 ' long in Degrees',
 ' orientation angles and SD in Degrees',
 ' velocity in Meter/Sec',
 ' position SD in Meters']

In [63]:
trajectory = np.array([line.strip().split()[:5] for line in data])
trajectory.shape

(500576, 5)

In [69]:
sample_size = 500

step = int(trajectory.shape[0] / sample_size)
downsample_ix = np.arange(0, trajectory.shape[0], step)

In [70]:
trajectory[downsample_ix]

array([['483583.06453', '0.000', '612017.243', '453188.735', '21.940'],
       ['483588.06955', '0.000', '612017.243', '453188.735', '21.940'],
       ['483593.07457', '0.000', '612017.243', '453188.735', '21.940'],
       ...,
       ['486075.56695', '9466.892', '613926.119', '446555.107', '11.393'],
       ['486080.57197', '9466.892', '613926.119', '446555.107', '11.393'],
       ['486085.57700', '9466.892', '613926.119', '446555.107', '11.393']],
      dtype='<U12')